In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel(r"C:\Users\karthick's\Desktop\Jupyter\HLB\Data.xlsx")
df.head()

,uniqueid,Bads,PAYMENT_LATEST_PAYMENT_METHOD,PAYMENT_AUTODEBIT,LOAN_AMOUNT,PAYMENT_MISSED_PAYMENTS_BUCKET,EMI_BUCKET,BANKS_MOBILEAPPUSAGE_DUR_BUCKET,PROFILE_CONTRACT_TENURE_BANK,CASA_TRANSACTION_AMOUNT_PERMONTH_BUCKET,CASA_TRNX_BUCKET,CASA_TRANS_DEBIT_BUCKET,CASA_TRANS_CREDIT_BUCKET,NUMBER_OF_CREDIT_PRODUCTS_WITH_BANK,NUMBER_OF_LIABILITIES_PRODUCTS_WITH_BANK,Age,enq012_bank1,enq012_bank2,enq012allbanks,enq006same bank
0,1,0,1.0,0,4,1,3.0,4,NaN,NaN,NaN,4,4,5,1.0,61,0,0,0,0
1,2,0,2.0,0,3,1,2.0,3,NaN,5.0,3.0,3,3,2,2.0,27,0,0,3,0
2,3,0,1.0,0,4,1,3.0,4,NaN,NaN,NaN,4,4,5,2.0,41,0,0,0,0
3,4,0,1.0,0,1,1,1.0,1,NaN,NaN,NaN,1,1,1,3.0,47,0,0,0,0
4,5,0,2.0,0,3,1,4.0,3,NaN,NaN,NaN,3,2,3,1.0,27,1,1,4,0


In [6]:
woe_vars_1 = df.rename(columns={"Bads": "Target"})
numlist = ['PAYMENT_AUTODEBIT','LOAN_AMOUNT','PAYMENT_MISSED_PAYMENTS_BUCKET']

f3 = []
for var in numlist:
    f = woe_vars_1.groupby(var, as_index=False).agg({"Target": ["count", "sum"]})
    f.columns = ['Buckets', 'N', 'EVENT']
    f['% of EVENT'] = np.maximum(f['EVENT'], 0.5) / f['EVENT'].sum()
    f['Non-EVENT'] = f['N'] - f['EVENT']
    f['% of Non-EVENT'] = np.maximum(f['Non-EVENT'], 0.5) / f['Non-EVENT'].sum()
    f['WoE'] = np.log(f['% of EVENT']/f['% of Non-EVENT'])
    f['IV'] = f['WoE'] * (f['% of EVENT'] - f['% of Non-EVENT'])
    f['var_name'] = np.resize(var,len(f))
    f2 = f[['var_name','Buckets','WoE','IV']]
    f3.append(f2)

woe_vars_2 = pd.concat(f3, ignore_index=True)

In [7]:
woe_vars_2

,var_name,Buckets,WoE,IV
0,PAYMENT_AUTODEBIT,0,0.146704,0.018393
1,PAYMENT_AUTODEBIT,1,-0.934075,0.117111
2,LOAN_AMOUNT,1,-0.290434,0.007598
3,LOAN_AMOUNT,2,-0.303781,0.020319
4,LOAN_AMOUNT,3,-0.092776,0.001840
5,LOAN_AMOUNT,4,0.128887,0.003817
6,LOAN_AMOUNT,5,0.345361,0.028755
7,PAYMENT_MISSED_PAYMENTS_BUCKET,1,-0.080228,0.005751
8,PAYMENT_MISSED_PAYMENTS_BUCKET,2,0.603569,0.026057
9,PAYMENT_MISSED_PAYMENTS_BUCKET,3,0.916247,0.019010


In [8]:
woe_vars_3 = pd.DataFrame(woe_vars_2.groupby('var_name')['IV'].sum())
woe_vars_3.columns = ['IV_2']
woe_vars_3.loc[(woe_vars_3['IV_2'] <  0.02), 'IV_importance'] = 'Not Good for prediction'
woe_vars_3.loc[(woe_vars_3['IV_2'] >= 0.02) & (woe_vars_3['IV_2'] < 0.1), 'IV_importance'] = 'Weak prediction'
woe_vars_3.loc[(woe_vars_3['IV_2'] >= 0.1 ) & (woe_vars_3['IV_2'] < 0.3), 'IV_importance'] = 'Medium predictive power'
woe_vars_3.loc[(woe_vars_3['IV_2'] >= 0.3 ) & (woe_vars_3['IV_2'] < 0.5), 'IV_importance'] = 'Strong predictive power'
woe_vars_3.loc[(woe_vars_3['IV_2'] >  0.5 ), 'IV_importance'] = 'Suspicious predictive power'

In [9]:
woe_vars_4 = woe_vars_2.merge(woe_vars_3, on='var_name',how='left')

In [11]:
transform_vars_list = df.columns.difference(['Target'])
transform_prefix = 'WOE_' # leave this value blank if you need replace the original column values

In [12]:
for var in transform_vars_list:
    small_df = woe_vars_4[woe_vars_4['var_name'] == var]
    transform_dict = dict(zip(small_df.Buckets,small_df.WoE))
    replace_cmd = ''
    replace_cmd1 = ''
    for i in sorted(transform_dict.items()):
        replace_cmd = replace_cmd + str(i[1]) + str(' if x <= ') + str(i[0]) + ' else '
        replace_cmd1 = replace_cmd1 + str(i[1]) + str(' if x == "') + str(i[0]) + '" else '
    replace_cmd = replace_cmd + '0'
    replace_cmd1 = replace_cmd1 + '0'
    if replace_cmd != '0':
        try:
            df[transform_prefix + var] = df[var].apply(lambda x: eval(replace_cmd))
        except:
            df[transform_prefix + var] = df[var].apply(lambda x: eval(replace_cmd1))

In [13]:
df.head()

,uniqueid,Bads,PAYMENT_LATEST_PAYMENT_METHOD,PAYMENT_AUTODEBIT,LOAN_AMOUNT,PAYMENT_MISSED_PAYMENTS_BUCKET,EMI_BUCKET,BANKS_MOBILEAPPUSAGE_DUR_BUCKET,PROFILE_CONTRACT_TENURE_BANK,CASA_TRANSACTION_AMOUNT_PERMONTH_BUCKET,...,NUMBER_OF_CREDIT_PRODUCTS_WITH_BANK,NUMBER_OF_LIABILITIES_PRODUCTS_WITH_BANK,Age,enq012_bank1,enq012_bank2,enq012allbanks,enq006same bank,WOE_LOAN_AMOUNT,WOE_PAYMENT_AUTODEBIT,WOE_PAYMENT_MISSED_PAYMENTS_BUCKET
0,1,0,1.0,0,4,1,3.0,4,NaN,NaN,...,5,1.0,61,0,0,0,0,0.128887,0.146704,-0.080228
1,2,0,2.0,0,3,1,2.0,3,NaN,5.0,...,2,2.0,27,0,0,3,0,-0.092776,0.146704,-0.080228
2,3,0,1.0,0,4,1,3.0,4,NaN,NaN,...,5,2.0,41,0,0,0,0,0.128887,0.146704,-0.080228
3,4,0,1.0,0,1,1,1.0,1,NaN,NaN,...,1,3.0,47,0,0,0,0,-0.290434,0.146704,-0.080228
4,5,0,2.0,0,3,1,4.0,3,NaN,NaN,...,3,1.0,27,1,1,4,0,-0.092776,0.146704,-0.080228
